# 循环神经网络从零开始实现
本节从零开始基于循环神经网络实现字符级语言模型。
## 读取数据集

In [1]:
import torch
import torch.nn as nn
import math
from d2l import torch as d2l
from torch.nn import functional as F

导入time machine数据集构建词表和数据集迭代器。

In [2]:
batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)

In [3]:
next(iter(train_iter))[0].shape

torch.Size([32, 35])

In [4]:
next(iter(train_iter))[1].shape

torch.Size([32, 35])

先回顾一下独热编码，简言之，将每个索引映射为相互不同的单位向量：假设词表中不同词元的数目为$N$（即`len(vocab)`），词元索引的范围为$0$到$N-1$。如果词元的索引是整数$i$，那么我们将创建一个长度为$N$的全$0$向量，并将第$i$处的元素设置为$1$。此向量是原始词元的一个独热向量。索引为$0$和$2$的独热向量如下所示：

In [5]:
F.one_hot(torch.tensor([0, 2]), len(vocab))

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]])

我们每次采样的(**小批量数据形状是二维张量：（批量大小，时间步数）。**)`one_hot`函数将这样一个小批量数据转换成三维张量，张量的最后一个维度等于词表大小（`len(vocab)`）。我们经常转换输入的维度，以便获得形状为**（时间步数，批量大小，词表大小）**的输出。这将使我们能够更方便地通过最外层的维度，一步一步地更新小批量数据的隐状态。

In [6]:
X = torch.arange(10).reshape((2, 5))
X

tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])

In [7]:
X.T

tensor([[0, 5],
        [1, 6],
        [2, 7],
        [3, 8],
        [4, 9]])

In [8]:
F.one_hot(X.T, len(vocab)).shape

torch.Size([5, 2, 28])

## 初始化模型参数
接下来我们初始化循环神经网络的模型参数，隐藏单元数`num_hidden`是一个可调的超参数。当训练语言模型时，输入输出来自相同的词表，因此，它们具有相同的维度，即词表的大小。

In [9]:
def get_params(vocab_size, num_hidden, device):
    num_inputs = num_outputs = vocab_size
    
    # 隐藏层参数
    W_xh = torch.normal(0, 0.01, (num_inputs, num_hiddens), device=device)
    W_hh = torch.normal(0, 0.01, (num_hiddens, num_hiddens), device=device)
    b_h = torch.zeros(num_hiddens, device=device)
    
    # 输出层参数
    W_hq = torch.normal(0, 0.01, (num_hiddens, num_outputs), device=device)
    b_q = torch.zeros(num_outputs, device=device)
    # 附加梯度
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.requires_grad_(True)
    return params

## 循环神经网络模型
为了定义循环神经网络模型，我们首先需要一个`init_rnn_state`函数在初始化时返回隐状态。这个函数的返回是一个全为零的张量，形状是（批量大小，隐藏单元数）。在后面的章节中我们将会遇到隐状态包含多个变量的情况， 而使用元组可以更容易地处理些。

In [10]:
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device),)

下面的`rnn`函数在一个时间步内计算隐状态和输出。循环神经网络模型通过`inputs`最外层的维度实现循环，以便逐时间步更新小批量数据的隐状态`H`。

In [11]:
def rnn(inputs, state, params):
    # inputs的形状：(时间步数量，批量大小，词表大小)
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        H = torch.tanh(torch.matmul(X, W_xh) 
                       + torch.matmul(H, W_hh)
                       + b_h)
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    # 输出的维度：(批量大小*steps, vocab_size)
    return torch.cat(outputs, dim=0), (H,)

定义了所有需要的函数之后，接下来我们创建一个类来包装这些函数，并存储从零开始实现的循环神经网络模型的参数：

In [12]:
class RNNModelScratch:
    """从零开始实现的循环神经网络模型"""
    def __init__(self, vocab_size, num_hiddens, device,
                    get_params, init_state, forward_fn):
        self.vocab_size = vocab_size
        self.num_hiddens = num_hiddens
        self.params = get_params(vocab_size, num_hiddens, device)
        self.init_state = init_state
        self.forward_fn = forward_fn
        
    def __call__(self, X, state):
        X = F.one_hot(X.T, self.vocab_size).type(torch.float32)
        return self.forward_fn(X, state, self.params)

    def begin_state(self, batch_size, device):
        return self.init_state(batch_size, self.num_hiddens, device)    

让我们检查输出是否具有正确的形状。 例如，隐状态的维数是否保持不变。

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
num_hiddens = 512

In [15]:
net = RNNModelScratch(len(vocab), num_hiddens, device, get_params,
                      init_rnn_state, rnn)

In [16]:
state = net.begin_state(X.shape[0],device)

In [17]:
Y, new_state = net(X.to(device), state)

输出形状是（（时间步数$\times$批量大小，词表大小）

In [18]:
Y.shape

torch.Size([10, 28])

In [19]:
len(new_state)

1

隐状态形状保持不变，即（批量大小，隐藏单元数）

In [20]:
new_state[0].shape

torch.Size([2, 512])

## 预测
首先定义预测函数来生成`prefix`之后的新字符，其中的`prefix`是一个用户提供的包含多个字符的字符串。在循环遍历`prefix`中的开始字符时，我们不断地将隐状态传递到下一个时间步，但是不生成任何输出。这被称为**预热期**,因为在此期间模型会自我更新隐状态，但不会进行预测。预热器结束后，隐状态的值通常比刚开始的初始值更适合预测。

In [22]:
def predict(prefix, num_preds, net, vocab, device):
    """在prefix后面生成新的字符"""
    state = net.begin_state(batch_size=1, device=device)
    outputs = [vocab[prefix[0]]]
    get_input = lambda: torch.tensor([outputs[-1]], device=device).reshape((1, 1))
    # 预热期
    for y in prefix[1:]:
        _, state = net(get_input(), state)
        outputs.append(vocab[y])
    
    for _ in range(num_preds):
        y, state = net(get_input(), state)
        outputs.append(int(y.argmax(dim=1).reshape(1)))
    
    return ''.join([vocab.idx_to_token[i] for i in outputs])

现在我们可以测试`predict_ch8`函数。我们将前缀指定为`time traveller `，并基于这个前缀生成10个后续字符。鉴于我们还没有训练网络，它会生成荒谬的预测结果。

In [23]:
predict('time traveller ', 10, net, vocab, device)

'time traveller bbaccccccc'

## 梯度剪裁
对于长度为$T$的序列，我们在迭代中计算这$T$个时间步上的梯度，将会在反向传播过程中产生长度为$\mathcal{O}(T)$的矩阵乘法链。当$T$较大时，它可能导致数值不稳定，例如可能导致梯度爆炸或梯度消失。因此，循环神经网络模型往往需要额外的方式来支持稳定训练。

对于梯度爆炸而言，一个流行的替代方案是通过将梯度$\mathbf{g}$投影回给定半径（例如$\theta$）的球来裁剪梯度$\mathbf{g}$。
如下式：

(**$$\mathbf{g} \leftarrow \min\left(1, \frac{\theta}{\|\mathbf{g}\|}\right) \mathbf{g}.$$**)

通过这样做，我们知道梯度范数永远不会超过$\theta$，并且更新后的梯度完全与$\mathbf{g}$的原始方向对齐。


下面我们定义一个函数来裁剪模型的梯度，模型是从零开始实现的模型或由高级API构建的模型。我们在此计算了所有模型参数的梯度的范数。

In [33]:
def grad_clipping(net, theta):
    """梯度裁剪"""
    if isinstance(net, nn.Module):
        params = [p for p in net.parameters() if p.requires_grad]
    
    else:
        params = net.params
    
    norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    if norm > theta:
        for param in params:
            param.grad[:] * theta / norm

## 训练


In [39]:
def train_epoch(net, train_iter, loss, updater, device, use_random_iter):
    """训练网络一个迭代周期"""
    state = None
    num_samples = 0
    loss_total = 0
    for X, y in train_iter:
        if state is None or use_random_iter:
            # 在第一次迭代或者使用随机抽样时初始化state
            state = net.begin_state(batch_size=X.shape[0], device=device)
        else:
            if isinstance(net, nn.Module) and not isinstance(state, tuple):
                # state对于GRU是一个张量
                state.detach_()
            else:
                # state 对于nn.LSTM或对于我们从零开始实现的模型是个张量
                for s in state:
                    s.detach_()
        y = y.T.reshape(-1)
        X, y = X.to(device), y.to(device)
        y_hat, state = net(X, state)
        l = loss(y_hat, y.long()).mean()
        if isinstance(updater, torch.optim.Optimizer):
            updater.zero_grad()
            l.backward()
            grad_clipping(net, 1)
            updater.step()
        else:
            l.backward()
            grad_clipping(net, 1)
            # 因为已经调用了mean函数
            updater(batch_size=1)
        loss_total += l * y.numel()
        num_samples += y.numel()
    return math.exp(loss_total / num_samples)

In [47]:
def train(net, train_iter, vocab, lr, num_epochs, device,
              use_random_iter=False):
    """训练模型（定义见第8章）"""
    loss = nn.CrossEntropyLoss()
    if isinstance(net, nn.Module):
        updater = torch.optim.SGD(net.parameters(), lr)
    else:
        updater = lambda batch_size: d2l.sgd(net.params, lr, batch_size)
    prediction = lambda prefix: predict(prefix, 50, net, vocab, device)
    # 训练和预测
    for epoch in range(num_epochs):
        ppl= train_epoch(
            net, train_iter, loss, updater, device, use_random_iter)
        if (epoch + 1) % 10 == 0:
            print(prediction('time traveller')) 
            print(f'困惑度 {ppl:.1f}')


In [48]:
num_epochs, lr = 200, 1
train(net, train_iter, vocab, lr, num_epochs, device)

time travellereetreetreetreetreetreetreetreetreetreetreetreetree
困惑度 1177917647421301125722144768.0
time travellereetteetteetteetteetteetteetteetteetteetteetteettee
困惑度 1107438023225415697180917760.0
time travellerwe bwe bwe bwe bwe bwe bwe bwe bwe bwe bwe bwe bwe
困惑度 603946194427383881641492480.0
time travellerelnselnselnselnselnselnselnselnselnselnselnselnsel
困惑度 18146961155863814898450432.0
time travellera sta sta sta sta sta sta sta sta sta sta sta sta 
困惑度 3313838390905698890687709184.0
time traveller ahn ahn ahn ahn ahn ahn ahn ahn ahn ahn ahn ahn a
困惑度 5920979534514001221934645248.0
time travellerae  ae  ae  ae  ae  ae  ae  ae  ae  ae  ae  ae  ae
困惑度 26475221664868023865664602112.0
time travelleroergoergoergoergoergoergoergoergoergoergoergoergoe
困惑度 683979650255906701712556032.0
time traveller cgo cgo cgo cgo cgo cgo cgo cgo cgo cgo cgo cgo c
困惑度 18479722442914788015943450624.0
time travellerrsdlrsdlrsdlrsdlrsdlrsdlrsdlrsdlrsdlrsdlrsdlrsdlrs
困惑度 25917981278372147945799680.0
time